In [1]:
import numpy as np
import pandas as pd
#импортируем нужные библиотеки

In [2]:
data = pd.read_csv('restaurant_customer_satisfaction.csv',sep=',')
data.sample(4)
#загрузка данных и вывод 4х произвольных строчек 
#данный датасет представляет собой набор данных посетителя и рейтинг его удовлетворенности рестораном
#Задача данного датасета является задачей обучения с учителем(классификация), т.к. мы имеем целевую переменную HighSatisfaction, мы хотим предсказать 
#эту переменную, которая показывает высоко ли удовлетворен посетитель, на основе других признаков имеющихся в датасете(возраст, доход, пол, ожидание и тд)

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,PreferredCuisine,TimeOfVisit,GroupSize,DiningOccasion,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,WaitTime,ServiceRating,FoodRating,AmbianceRating,HighSatisfaction
947,1601,65,Female,55794,Rarely,45.286599,Chinese,Lunch,8,Business,Takeaway,0,1,1,4.016771,1,5,5,0
904,1558,45,Female,45842,Monthly,14.005649,Italian,Breakfast,7,Celebration,Takeaway,1,1,1,7.843588,1,3,4,0
147,801,52,Male,111967,Rarely,137.747655,American,Lunch,8,Casual,Dine-in,0,1,0,20.887859,2,5,4,0
790,1444,60,Male,34207,Monthly,64.054450,Italian,Breakfast,7,Business,Dine-in,0,1,1,36.535718,1,3,4,0


In [3]:
data.info()
#предварительный анализ датасета
#общая информация о датасете
#отсюда можем сделать вывод о том, что у нас 2 столбца типа float, 11 столбцов-int, 6 столбцов-object и их нам придется в будущем перекодировать в числовые значения
#в датасете 1500 строк
#в каждой колонке 1500 непустых значений, что значит, что в датасете нет пустых ячеек

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerID            1500 non-null   int64  
 1   Age                   1500 non-null   int64  
 2   Gender                1500 non-null   object 
 3   Income                1500 non-null   int64  
 4   VisitFrequency        1500 non-null   object 
 5   AverageSpend          1500 non-null   float64
 6   PreferredCuisine      1500 non-null   object 
 7   TimeOfVisit           1500 non-null   object 
 8   GroupSize             1500 non-null   int64  
 9   DiningOccasion        1500 non-null   object 
 10  MealType              1500 non-null   object 
 11  OnlineReservation     1500 non-null   int64  
 12  DeliveryOrder         1500 non-null   int64  
 13  LoyaltyProgramMember  1500 non-null   int64  
 14  WaitTime              1500 non-null   float64
 15  ServiceRating        

In [4]:
data.describe(include='object')
#функция для анализа строковых столбцов
#анализируем строковые столбцы: отсюда видно, что большинство посетителей женщины(759), большинство посещают ресторан еженедельно(606),
#большинство любят итальянскую кухню(325), большинство посещают за завтраком ресторан(506), большинство ходят по праздникам(519),и едят в ресторане(751)
#также с помощью этой функции можно узнать количество уникальных значений в столбцах

,Gender,VisitFrequency,PreferredCuisine,TimeOfVisit,DiningOccasion,MealType
count,1500,1500,1500,1500,1500,1500
unique,2,4,5,3,3,2
top,Female,Weekly,Italian,Breakfast,Celebration,Dine-in
freq,759,606,325,506,519,751


In [5]:
data.describe()
#функция для анализа нестроковых столбцов
#С помощью этой ф-ции можно сделать вывод, что например, средний возраст посетителя 43, что четверть посетителей имеют возраст не более 31, 
#половина-не более 44, 3 четверти не более 57
#важными показателями, на мой взгляд, будут являться то, что в 3х столбцах, отвечающающих за удовлетворенность, четверть удовлетворены не более, 
#чем на 2 балла, половина-не более, чем 3 балла, 3 четверти-не более, чем на 4 балла, уже отсюда понимаем, что распределение равномерное, и у ресторана
#нет перекоса в хорошую, или плохую сторону-он средний
#также можем сказать, что менее чем 3 четверти имеют карту лояльности(т.е.предпочитают скидки),оформили заказ доставкой, и сделали онлайн резерв
#также видно, что среднее время ожидания 30 мин, что является обычным временем ожидания для ресторанов
#ресторан посещают люди, со средним доходом 38 000 долларов,(средний доход в США на 2023г - 59 000 долларов), отсюда можно сделать вывод, что данный 
#ресторан могут посещать люди любого достатка, а соответственно в этой выборке учтены отзывы людей многих классов людей

,CustomerID,Age,Income,AverageSpend,GroupSize,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,WaitTime,ServiceRating,FoodRating,AmbianceRating,HighSatisfaction
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,1403.500000,43.832000,85921.890000,105.659004,5.035333,0.296667,0.405333,0.480000,30.163550,3.044000,2.997333,2.987333,0.134000
std,433.157015,14.967157,38183.051749,52.381849,2.558864,0.456941,0.491120,0.499766,17.214184,1.423405,1.418920,1.450716,0.340766
min,654.000000,18.000000,20012.000000,10.306127,1.000000,0.000000,0.000000,0.000000,0.001380,1.000000,1.000000,1.000000,0.000000
25%,1028.750000,31.750000,52444.000000,62.287907,3.000000,0.000000,0.000000,0.000000,15.235423,2.000000,2.000000,2.000000,0.000000
50%,1403.500000,44.000000,85811.000000,104.626408,5.000000,0.000000,0.000000,0.000000,30.044055,3.000000,3.000000,3.000000,0.000000
75%,1778.250000,57.000000,119159.250000,148.649330,7.000000,1.000000,1.000000,1.000000,45.285649,4.000000,4.000000,4.000000,0.000000
max,2153.000000,69.000000,149875.000000,199.973527,9.000000,1.000000,1.000000,1.000000,59.970762,5.000000,5.000000,5.000000,1.000000


In [6]:
data['Gender'].value_counts()
#начинаем обработку текстовых и категориальных признаков
#уникальные значения столбца Gender с частой

Gender
Female    759
Male      741
Name: count, dtype: int64

In [7]:
data.Gender.map({'Female':0,'Male':1})

0       1
1       1
2       0
3       1
4       0
       ..
1495    1
1496    0
1497    1
1498    1
1499    1
Name: Gender, Length: 1500, dtype: int64

In [8]:
data['Gender']=data.Gender.map({'Female':0,'Male':1})
#первый строковый столбец, посколькую значения всего 2 и они понятно чем отличаются, можем заменить числовыми 0 и 1

In [9]:
data.head(2)

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,PreferredCuisine,TimeOfVisit,GroupSize,DiningOccasion,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,WaitTime,ServiceRating,FoodRating,AmbianceRating,HighSatisfaction
0,654,35,1,83380,Weekly,27.829142,Chinese,Breakfast,3,Business,Takeaway,0,1,1,43.523929,2,5,4,0
1,655,19,1,43623,Rarely,115.408622,American,Dinner,1,Casual,Dine-in,0,0,0,57.524294,5,5,3,0


In [10]:
data['VisitFrequency'].value_counts()
#уникальные значения столбца VisitFrequency с частой

VisitFrequency
Weekly     606
Monthly    428
Rarely     313
Daily      153
Name: count, dtype: int64

In [11]:
data['VisitFrequency']=data.VisitFrequency.map({'Weekly':3,'Monthly':2,'Rarely':1,'Daily':4})
#второй строковый столбец, где значения хорошо сравнимы, ежедневно/ежемесячно.., чем меньше число, тем реже посещаемость данного клиента
#поэтому можем заменить на числовые значения

In [12]:
data.sample(4)

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,PreferredCuisine,TimeOfVisit,GroupSize,DiningOccasion,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,WaitTime,ServiceRating,FoodRating,AmbianceRating,HighSatisfaction
845,1499,61,0,60147,3,66.170675,American,Dinner,2,Celebration,Dine-in,1,1,0,9.872375,2,4,4,1
1141,1795,38,1,74170,2,107.566637,Mexican,Lunch,1,Casual,Dine-in,1,0,0,23.170861,4,2,3,0
302,956,66,1,65207,2,121.371064,Chinese,Dinner,7,Casual,Dine-in,1,1,0,50.188765,1,5,1,0
1407,2061,50,1,84448,3,164.644164,American,Breakfast,3,Celebration,Takeaway,0,1,0,57.145126,3,2,5,0


In [13]:
data['PreferredCuisine'].value_counts()
#здесь попался столбец, с несравнимыми значениями(виды кухни) и их всего 5, поэтому разделим их на отдельные столбцы со значениями true/false

PreferredCuisine
Italian     325
Chinese     310
Mexican     299
Indian      296
American    270
Name: count, dtype: int64

In [14]:
pd.get_dummies(data['PreferredCuisine'],prefix='PC')
#делаем отдельные столбцы со значениями true/false с префиксом PC

,PC_American,PC_Chinese,PC_Indian,PC_Italian,PC_Mexican
0,False,True,False,False,False
1,True,False,False,False,False
2,True,False,False,False,False
3,False,False,True,False,False
4,False,True,False,False,False
...,...,...,...,...,...
1495,True,False,False,False,False
1496,False,False,False,True,False
1497,True,False,False,False,False
1498,False,False,False,True,False


In [15]:
data=pd.concat([data,pd.get_dummies(data['PreferredCuisine'],prefix='PC')],axis=1)
#добавляем эти столбцы к исходному датасету

In [16]:
data.sample(4)

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,PreferredCuisine,TimeOfVisit,GroupSize,DiningOccasion,...,WaitTime,ServiceRating,FoodRating,AmbianceRating,HighSatisfaction,PC_American,PC_Chinese,PC_Indian,PC_Italian,PC_Mexican
25,679,40,0,38099,3,118.814236,Chinese,Breakfast,1,Casual,...,20.895781,3,2,1,0,False,True,False,False,False
1362,2016,60,1,93622,3,128.902592,Indian,Dinner,8,Business,...,35.198699,2,4,4,0,False,False,True,False,False
881,1535,23,0,95587,3,94.850658,Mexican,Breakfast,9,Business,...,15.609551,5,5,1,1,False,False,False,False,True
837,1491,46,0,78259,3,86.403799,Chinese,Breakfast,4,Casual,...,27.630489,5,1,3,1,False,True,False,False,False


In [17]:
data.drop('PreferredCuisine',axis=1,inplace=True)
#удаляем столбец PreferredCuisine

In [18]:
data['TimeOfVisit'].value_counts()

TimeOfVisit
Breakfast    506
Lunch        502
Dinner       492
Name: count, dtype: int64

In [19]:
data['TimeOfVisit']=data.TimeOfVisit.map({'Breakfast':1,'Lunch':2,'Dinner':3})
#перекодировали еще один столбец со сравнимыми значениями

In [20]:
data['MealType'].value_counts()

MealType
Dine-in     751
Takeaway    749
Name: count, dtype: int64

In [21]:
data['MealType']=data.MealType.map({'Dine-in':0,'Takeaway':1})
#перекодировали еще один столбец со сравнимыми значенями, где всего 2 значения

In [22]:
data['DiningOccasion'].value_counts()
#уникальные значения столбца Gender с частой

DiningOccasion
Celebration    519
Business       500
Casual         481
Name: count, dtype: int64

In [23]:
data=pd.concat([data,pd.get_dummies(data['DiningOccasion'],prefix='DO')],axis=1)
#попался еще один столбец с несравнимыми значениями и их всего 4, поэтому выделяем их них отдельные столбцы

In [24]:
data.drop('DiningOccasion',axis=1,inplace=True)

In [25]:
data.sample(5)

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,TimeOfVisit,GroupSize,MealType,OnlineReservation,...,AmbianceRating,HighSatisfaction,PC_American,PC_Chinese,PC_Indian,PC_Italian,PC_Mexican,DO_Business,DO_Casual,DO_Celebration
437,1091,55,1,125245,1,115.461597,3,5,1,1,...,4,0,False,True,False,False,False,False,True,False
382,1036,69,0,96870,2,108.858751,2,3,0,0,...,2,0,False,True,False,False,False,True,False,False
184,838,52,1,100057,2,127.662245,1,7,0,0,...,1,0,False,False,True,False,False,True,False,False
614,1268,37,0,122112,1,69.369328,3,8,0,0,...,2,0,True,False,False,False,False,True,False,False
595,1249,20,0,129285,3,134.605581,3,7,1,0,...,1,0,True,False,False,False,False,False,True,False


In [26]:
data.info()
#в результате было 19 столбцов, а получилось 25, также мы заменили все строковые столбцы,числовыми или булевскими

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerID            1500 non-null   int64  
 1   Age                   1500 non-null   int64  
 2   Gender                1500 non-null   int64  
 3   Income                1500 non-null   int64  
 4   VisitFrequency        1500 non-null   int64  
 5   AverageSpend          1500 non-null   float64
 6   TimeOfVisit           1500 non-null   int64  
 7   GroupSize             1500 non-null   int64  
 8   MealType              1500 non-null   int64  
 9   OnlineReservation     1500 non-null   int64  
 10  DeliveryOrder         1500 non-null   int64  
 11  LoyaltyProgramMember  1500 non-null   int64  
 12  WaitTime              1500 non-null   float64
 13  ServiceRating         1500 non-null   int64  
 14  FoodRating            1500 non-null   int64  
 15  AmbianceRating       

In [27]:
data[data.isnull().any(axis=1)]
#в функции data.info() и так видно, что пустых ячеек нет, но проверяем это еще раз, строки такие не вывелись, следовательно, в этом пункте ничего 
#не нужно делать

,CustomerID,Age,Gender,Income,VisitFrequency,AverageSpend,TimeOfVisit,GroupSize,MealType,OnlineReservation,...,AmbianceRating,HighSatisfaction,PC_American,PC_Chinese,PC_Indian,PC_Italian,PC_Mexican,DO_Business,DO_Casual,DO_Celebration


In [28]:
from sklearn.preprocessing import StandardScaler
#импортируем библиотеку для масштабирования

In [29]:
features_to_scale = ['Age', 'Income', 'AverageSpend', 'GroupSize', 'WaitTime', 'ServiceRating', 'FoodRating', 'AmbianceRating']
#выбрали признаки для масштабирования

In [30]:
scaler = StandardScaler()
#создали объект StandardScaler

In [31]:
scaled_data = scaler.fit_transform(data[features_to_scale])
#масштабирование

In [32]:
scaled_df = pd.DataFrame(scaled_data, columns=['Age1', 'Income1', 'AverageSpend1', 'GroupSize1', 'WaitTime1', 'ServiceRating1', 'FoodRating1', 'AmbianceRating1'])
#создаем новый датафрейм с отмасштабированными данными

In [33]:
scaled_df.sample(5)

,Age1,Income1,AverageSpend1,GroupSize1,WaitTime1,ServiceRating1,FoodRating1,AmbianceRating1
1275,0.479075,0.050461,-0.121055,-0.404742,-0.286092,0.671853,0.001880,1.387824
125,0.011228,-1.242559,-1.793980,-0.013813,-0.198881,-1.436472,0.706876,0.008734
50,0.278569,0.543567,1.734050,-1.186599,1.272941,-1.436472,-1.408112,0.698279
545,-1.124971,-1.395532,-1.718320,-1.186599,-0.383148,-1.436472,1.411872,-0.680811
717,0.679581,0.334006,-0.739818,-0.404742,1.167080,-1.436472,-0.703116,-0.680811


In [34]:
data = pd.concat([data, scaled_df], axis=1) 
#объединяем датафреймы

In [35]:
data.drop(columns=features_to_scale, inplace=True)
#удаляем столбцы из исходного датафрейма

In [36]:
data.sample(5)

,CustomerID,Gender,VisitFrequency,TimeOfVisit,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,HighSatisfaction,PC_American,...,DO_Casual,DO_Celebration,Age1,Income1,AverageSpend1,GroupSize1,WaitTime1,ServiceRating1,FoodRating1,AmbianceRating1
1228,1882,0,2,3,1,0,0,1,0,False,...,False,True,-1.525982,0.187583,-0.933111,0.377116,0.504019,-1.436472,1.411872,-1.370355
727,1381,0,2,3,0,1,1,0,0,False,...,False,False,-0.456618,-1.037924,0.590923,1.549902,0.325537,0.671853,0.706876,1.387824
242,896,0,2,1,0,0,1,1,0,False,...,True,False,0.211734,0.672017,-0.719318,-1.186599,-1.007195,-0.733697,0.001880,0.008734
786,1440,1,2,1,1,0,1,1,0,False,...,True,False,-0.857630,0.715428,1.061367,-0.404742,1.205673,1.374628,0.706876,0.698279
1254,1908,0,3,2,0,0,0,0,0,False,...,True,False,-1.459147,1.406593,1.169948,-0.013813,-0.551672,1.374628,0.706876,1.387824


In [37]:
columns_to_convert = ['PC_American', 'PC_Chinese', 'PC_Indian', 'PC_Italian', 'PC_Mexican']
data[columns_to_convert] = data[columns_to_convert].astype(int)

In [38]:
columns_to_convert = ['DO_Business', 'DO_Casual', 'DO_Celebration']
data[columns_to_convert] = data[columns_to_convert].astype(int)

In [39]:
data.sample(5)

,CustomerID,Gender,VisitFrequency,TimeOfVisit,MealType,OnlineReservation,DeliveryOrder,LoyaltyProgramMember,HighSatisfaction,PC_American,...,DO_Casual,DO_Celebration,Age1,Income1,AverageSpend1,GroupSize1,WaitTime1,ServiceRating1,FoodRating1,AmbianceRating1
228,882,0,2,2,1,0,0,1,0,1,...,1,0,-0.122442,-1.481384,-1.332037,-0.795670,-0.184472,-0.030922,0.001880,1.387824
1095,1749,1,3,3,0,0,1,0,0,0,...,0,1,1.481604,1.475862,0.047041,0.377116,-0.481340,0.671853,-1.408112,-1.370355
1229,1883,1,1,2,0,1,0,1,0,0,...,0,0,1.013757,0.990668,-0.694645,-0.404742,-0.478369,0.671853,0.001880,0.008734
1384,2038,1,3,2,1,1,0,1,0,0,...,1,0,1.347933,-1.225950,-0.922791,-1.577528,0.636924,-0.030922,0.706876,0.008734
1498,2152,1,3,3,0,1,1,0,1,0,...,1,0,-1.325476,-1.518245,1.732182,0.377116,-0.344919,-0.733697,-0.703116,0.698279


In [40]:
df=data.copy()